In [2]:
import numpy as np
from Models.utils import PrimalDualSolver, ProjectedGradientDescent

class SDL:
    """
    Supervised Dictionary Learning (SDL) Class.

    Combines sparse coding with supervised learning by jointly learning:
    - A dictionary `D` for sparse representation.
    - A linear model `(theta, b)` for predicting labels.
    """

    def __init__(self, n_iter=1000,
                 lamnda0=0.01,
                 lambda1=0.01,
                 lambda2=0.01,
                 lr_D=0.01,
                 lr_theta=0.01,
                 lr_alpha=0.01):
        self.n_iter = n_iter
        self.lamnda0 = lamnda0
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.lr_D = lr_D
        self.lr_theta = lr_theta
        self.lr_alpha = lr_alpha

    def objective(self, X, y, D, theta, alpha):
        """Computes the objective function value."""
        objective = 0
        for i in range(X.shape[0]):
            xi, yi, ai = X[i], y[i], alpha[i]
            loss_dict = np.linalg.norm(xi - D @ ai)**2
            loss_class = np.linalg.norm(yi - theta @ ai)**2
            sparse_penalty = self.lambda1 * np.linalg.norm(ai, 1)
            objective += loss_dict + loss_class + sparse_penalty
        return objective

    def solve_alpha(self, X, y, D, theta, b):
        """
        Optimizes sparse codes `alpha` for fixed `D` and `theta`.
        We can have here a explicit expression of our gradient regarding
        to alpha, so we can use a proximal gradient descent to optimize it.
        """
        n_samples, n_features = X.shape
        alpha = np.zeros((n_samples, n_features))

        for i in range(n_samples):
            x_i = X[i]
            y_i = y[i]

            solver = PrimalDualSolver(
                theta=theta, b=b, x_i=x_i, y_i=y_i, D=D,
                lambda_0=self.lamnda0, lambda_1=self.lambda1,
                lambd=0.01, mu=1.0
            )
            # Solve the problem
            x0 = np.random.randn(n_features)  # Random initialization
            alpha_opt, _ = solver.solve(x0)

            alpha[i] = alpha_opt

        return alpha

    def solve_D_theta(self, alpha_opt, X, y, D_opt, theta_opt, b):
        """
        Updates `D` and `theta` given the optimal `alpha`.
        Do a projective gradient descent.
        """
        pgd = ProjectedGradientDescent(
            D_init=D_opt, theta_init=theta_opt,
            b=b, x=X, y=y, alphas=alpha_opt,
            lambda_0=self.lamnda0,
            lambda_1=self.lambda1, lambda_2=self.lambda2,
            lr=self.lr_D, max_iter=self.n_iter
        )
        D_opt, theta_opt, b_opt, _ = pgd.optimize()
        return D_opt, theta_opt, b_opt

    def fit(self, X, y):
        """Fits the model to the data."""
        n_samples, n_features = X.shape
        self.n_components = n_features
        D_opt = np.random.randn(n_features, self.n_components)
        D_opt /= np.linalg.norm(D_opt, axis=0)
        theta_opt = np.zeros(self.n_components)
        b_opt = 0

        for i in range(self.n_iter):
            alpha_opt = self.solve_alpha(X, y, D_opt, theta_opt, b_opt)
            print(" alpha_opt: ", alpha_opt)
            D_opt, theta_opt, b_opt = self.solve_D_theta(alpha_opt,
                                                            X,
                                                            y,
                                                            D_opt,
                                                            theta_opt,
                                                            b_opt)
            print(" D_opt: ", D_opt)
            print(" theta_opt: ", theta_opt)
            print(" b_opt: ", b_opt)
            print('Loss at iteration {}: {}'.format(i, self.objective(X, y, D_opt, theta_opt, alpha_opt)))

        self.alpha = alpha_opt
        self.D = D_opt
        self.theta = theta_opt
        self.b = b_opt

    def predict(self, X):
        """Predicts labels for input data `X`."""
        return self.theta @ self.alpha.T + self.b

    def score(self, X, y):
        """Computes classification accuracy."""
        y_pred = self.predict(X)
        return np.mean(np.round(y_pred) == y)

In [ ]:

# Generate synthetic data
np.random.seed(42)
n_samples, n_features, n_components = 100, 20, 10
X = np.random.randn(n_samples, n_features)  # Input data
true_D = np.random.randn(n_features, n_components)  # True dictionary
true_alpha = np.random.randn(n_samples, n_components)  # Sparse codes
true_theta = np.random.randn(n_components)  # Linear model weights
b_true = 0.5  # Bias term

# Generate labels based on the linear model
y = X @ true_D @ true_theta + b_true + 0.1 * np.random.randn(n_samples)
y = np.round(y)  # Binary classification labels (0 or 1)

# Initialize the SDL model
sdl = SDL(
    n_iter=10,
    lamnda0=0.01,
    lambda1=0.1,
    lambda2=0.1,
    lr_D=0.01,
    lr_theta=0.01,
    lr_alpha=0.01
)

# Fit the model to the data
sdl.fit(X, y)

# Evaluate the model
accuracy = sdl.score(X, y)
print(f"Classification Accuracy: {accuracy:.2f}")

# Predictions
y_pred = sdl.predict(X)
print(f"Predicted Labels: {np.round(y_pred)}")

# print the true


 alpha_opt:  [[-0.47889571 -0.448591    0.         ... -0.          0.
  -1.16095885]
 [-0.          0.         -0.75542079 ...  0.          0.04441189
  -0.14901073]
 [ 0.          0.          0.14127548 ...  0.15692831 -0.
  -0.        ]
 ...
 [ 0.          0.         -0.         ... -0.         -0.71442631
   0.28583318]
 [ 1.02039323 -0.          0.         ... -0.         -0.14887817
   0.        ]
 [ 0.         -0.68942402 -0.         ...  0.         -0.
   0.7388329 ]]
 D_opt:  [[-7.49833494e-02  2.09636320e-02  8.92188785e-02 -5.14006958e-02
  -1.61682679e-02  1.76309395e-02  5.44613081e-03 -4.30657656e-02
   6.39758243e-02  9.05937219e-02  9.73002041e-03 -3.92843739e-02
  -6.08627202e-02  1.23522127e-02  5.51324038e-02  2.07547504e-02
   2.26016472e-03 -3.53429998e-02 -6.79211499e-02  4.62630170e-02]
 [ 6.23624824e-02 -7.18093886e-02  6.65547089e-02  2.03090356e-02
   5.09068175e-02 -6.07968968e-02 -9.36066912e-02 -3.76371470e-03
   3.14047430e-02  8.41902823e-02  3.50479955e-